<a href="https://colab.research.google.com/github/GuerrSim96/GuerrSim96/blob/main/Diagrams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install upsetplot -q
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # Suppress pandas FutureWarnings
import matplotlib.pyplot as plt
import zipfile, os, shutil
from matplotlib import pyplot as plt
from matplotlib_venn import venn3, venn3_circles
import seaborn as sns
from upsetplot import UpSet
from google.colab import files
uploaded = files.upload() # Upload zip files of 3 group
  # @title Extract file zip and move csv files in the working directory
list_dir = os.listdir()
for file_zip in list_dir:
    if file_zip.endswith('.zip'):
        with zipfile.ZipFile(file_zip, 'r') as zip_ref:
            zip_ref.extractall()
# Rename csv files
csv_name_list = []
cwd = os.getcwd()
for root, dirs, files in os.walk(cwd):
    for csv_file in files:
        if csv_file == 'proteins.csv':
            root_name = os.path.basename(root)
            csv_name = f'{root_name}_proteins.csv'
            csv_name_list.append(csv_name)
            departure, destination = os.path.join(root, csv_file), os.path.join(cwd, csv_name)
            shutil.move(departure, destination)

In [ ]:
csv_name_list = ['Cormey_v2_SPIDER_CMAC_GL_proteins.csv', 'Cormey_v2_SPIDER_CMBE_GL_proteins.csv', 'Cormey_v2_SPIDER_CMPA_GL_proteins.csv']

In [ ]:
title = 'Qualitative comparisons among C. Meyendorffii ecotype proteomes' #@param {"type":"string"}
group_1 = "A. calycularis" # @param {"type":"string"}
group_2 = "B. europea" # @param {"type":"string"}
group_3 = "P. axinellae" # @param {"type":"string"}
# Creation of DataFrame from csv file
df_1, df_2, df_3 = pd.read_csv(csv_name_list[0]), pd.read_csv(csv_name_list[1]), pd.read_csv(csv_name_list[2])
# Get list of protein accession name without repetition
set_1, set_2, set_3 = set(df_1['Accession'].values.flatten()), set(df_2['Accession'].values.flatten()), set(df_3['Accession'].values.flatten())
num_set_1, num_set_2, num_set_3 = len(set_1), len(set_2), len(set_3) # Number of the unique elements
# Creation of intersection between groups
I_1_2, I_1_3, I_2_3 = set_1.intersection(set_2), set_1.intersection(set_3), set_2.intersection(set_3)
num_I_1_2, num_I_1_3, num_I_2_3 = len(I_1_2), len(I_1_3), len(I_2_3) # Number of intersection elements
se = I_1_2.intersection(set_3) # List of elements shared by all the groups
num_se = len(se) # Number of elements shared by all the groups
# Diffetence between shared elements and intersection groups
D_I_1_2 = I_1_2.difference(se)
num_D_I_1_2 = len(D_I_1_2)
D_I_1_3 = I_1_3.difference(se)
num_D_I_1_3 = len(D_I_1_3)
D_I_2_3 = I_2_3.difference(se)
num_D_I_2_3 = len(D_I_2_3)
# Elements owned only by one group
U_I_1 = I_1_2.union(I_1_3)
D_1 = set_1.difference(U_I_1)
num_D_1 = len(D_1)
U_I_2 = I_1_2.union(I_2_3)
D_2 = set_2.difference(U_I_2)
num_D_2 = len(D_2)
U_I_3 = I_1_3.union(I_2_3)
D_3 = set_3.difference(U_I_3)
num_D_3 = len(D_3)

<h1> Venn diagram </h1>

In [ ]:
# @title Venn diagram for 3 groups
# Plot Venn
plt.figure(figsize=(4,4))
plt.title(title)
plt.tight_layout()
venn3(subsets = (num_D_1, num_D_2, num_D_I_1_2, num_D_3, num_D_I_1_3, num_D_I_2_3, num_se),
      set_labels = (group_1, group_2, group_3),
      set_colors = ('c', 'y', 'm'), alpha=0.7)
venn3_circles(subsets = (num_D_1, num_D_2, num_D_I_1_2, num_D_3, num_D_I_1_3, num_D_I_2_3, num_se), linewidth = 0.7, color = "black", linestyle = "dashed")
# Save figure
plt.savefig('Venn3_diagram.svg', format = 'svg', bbox_inches='tight')
plt.show()

<h1>UpSet diagram</i>

In [ ]:
# @title From UpSetPlot
'''
https://github.com/jnothman/UpSetPlot
https://upsetplot.readthedocs.io/en/latest/auto_examples/plot_diabetes.html
'''
df_1[group_1], df_2[group_2], df_3[group_3] = True, True, True
df_1 = df_1.sort_values(by = ['Accession', "-10lgP"], kind = 'stable')
df_1_drop = df_1.drop_duplicates(subset = 'Accession', keep = 'last')
df_2 = df_2.sort_values(by = ['Accession', "-10lgP"], kind = 'stable')
df_2_drop = df_2.drop_duplicates(subset = 'Accession', keep = 'last')
df_3 = df_3.sort_values(by = ['Accession', "-10lgP"], kind = 'stable')
df_3_drop = df_3.drop_duplicates(subset = 'Accession', keep = 'last')
df_drop_merge = df_1_drop.merge(df_2_drop.merge(df_3_drop, on='Accession', how='outer'), on='Accession', how='outer')
df_drop_merge[group_1] = df_drop_merge[group_1].fillna(False)
df_drop_merge[group_2] = df_drop_merge[group_2].fillna(False)
df_drop_merge[group_3] = df_drop_merge[group_3].fillna(False)
# Make sets index for the data frame
df_drop_merge_upset = df_drop_merge.set_index([group_1, group_2, group_3])
lst_features = ['-10lgP', 'Avg. Mass', 'Coverage (%)']
for feature in lst_features:
  df_drop_merge_upset[feature] = df_drop_merge_upset[feature].fillna(
      df_drop_merge_upset[f'{feature}_x']).fillna(df_drop_merge_upset[f'{feature}_y'])

In [ ]:
'''
https://seaborn.pydata.org/generated/seaborn.catplot.html
'''
# Plot UpSet diagram
plt.figure(figsize=(40,40))
plt.tight_layout()
upset_plot = UpSet(df_drop_merge_upset, subset_size='count')
# Aggiungere questi sotto plot al UpSet plot
upset_plot.add_catplot(value='Avg. Mass', kind='violin', color='lightblue', inner=None)
upset_plot.add_catplot(value='Coverage (%)', kind='box', color='pink')
upset_plot.add_catplot(value='-10lgP', kind='swarm', size=0.3, color='black')
upset_plot.plot()
plt.savefig('UpSet_plot.svg', format = 'svg')
plt.show()

In [ ]:
# @markdown Alternative Method to produce an UpSet plot
alt = False # @param {"type":"boolean"}
if alt:
  from upsetplot import from_memberships
  lst = [[],[group_1],[group_2],[group_3],[group_1, group_2],[group_1, group_3],[group_2, group_3],[group_1, group_2, group_3]]
  data = [0 ,num_D_1, num_D_2, num_D_3, num_D_I_1_2, num_D_I_1_3, num_D_I_2_3, num_se]
  # Plot UpSet diagram
  plt.tight_layout()
  upset_data = from_memberships(lst, data)
  upset_plot = UpSet(upset_data).plot() # Upset plot
  plt.show()
else:
  url = 'https://community.wappler.io/uploads/default/original/3X/e/d/ed19b79f21d49e0b3a342d10c01e990e2bd316cb.png'
  from PIL import Image
  import requests
  im = Image.open(requests.get(url, stream=True).raw)
  display(im)